# A4: Model Selection, Cross-validation, Confidence Intervals [ __ /70 marks]

<hr>
<img src= "https://media.pri.org/s3fs-public/styles/open_graph/public/photos/2014-May/nasa_firemap_2014_05_12.jpg?itok=V3d6c0dx" width=500>
<hr>

In this assignment we will compare 3 different models (and select one) on a modified version of the ["forest fires"](https://archive.ics.uci.edu/ml/datasets/Forest+Fires) dataset. Specifically, given some input features (temperature, relative humidity, etc.) and an output y (`area`) we wish to build models, select a particular model, and make predictions on unseen data. We also want to bound our prediction with a 95% confidence interval (CI); for this confidence interval we will use the Central Limit Theorem (CLT).

## Before you start...
* see relevant lecture code (`L5_CF.ipynb`, `L4_CF.ipynb`)

## Before you submit...
* restart the kernel, then re-run the whole notebook to ensure no errors

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import make_scorer
from sklearn.base import BaseEstimator, TransformerMixin
%matplotlib inline

## Question 1.1 [ _ /4 marks]

Read the file `ffd.csv` into a dataframe. Display the first 5 rows of this dataframe. 

In [2]:
# Read ffd.csv into a dataframe [ /2 marks] 
df = pd.read_csv('ffd.csv')

# Display the first 5 rows of the dataframe[ /2 marks]
display(df.head())

,longitude,latitude,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


## Question 1.2 [ _ /5 marks]

Let's convert all categorical data into numerical data using `get_dummies`. Before that, though, we should specify this data as *Categorical*; this has been done for you for the "month" column (you will need to do the same for the "day" column). Display the first 5 rows of your new dataframe.

In [3]:
df['month'] = pd.Categorical(df.month, categories=['jan','feb','mar', 'apr','may','jun','jul','aug','sep','oct','nov','dec'])

# Set non-numerical data as Categorical [ /2 marks]
df['day'] = pd.Categorical(df.day, categories=['sun','mon', 'tue','wed','thu','fri','sat'])

# Use "get_dummies" to convert categorical data to numerical data (you can set "drop_first=True") [ /2 marks]                      
df = pd.get_dummies(df, drop_first=True)

# Display first 5 rows of the dataframe [ /1 mark]
display(df.head())

,longitude,latitude,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,month_sep,month_oct,month_nov,month_dec,day_mon,day_tue,day_wed,day_thu,day_fri,day_sat
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,0,0,0,0,1,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,1,0,0,0,1,0,0,0,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,1,0,0,0,0,0,0,0,1
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,0,0,0,0,1,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,0,0,0,0,0,0


## Question 1.3 [ _ /4 marks]

Let's use **mean squared error** as our error function. We will use this to evaluate and compare our models. Write a function called `mse` with arguments `y` and `ypr`(predicted y) which returns the mean squared error. Recall the formula for MSE below:

$$ MSE = \frac{1}{n} \sum_{i=1}^{n}  \left( \hat{y_{i}}-y_{i}\right)^{2} $$

In [4]:
# Define the MSE function [ /4 marks]
def mse(y,ypr):
    return np.mean((y-ypr)**2)

## Question 1.4: [ _ /6 marks]

Let's now prepare our dataset by splitting it into training and test data. Use sklearn's `train_test_split` (`test_size=0.5`, `random_state=0`). 

In [5]:
# Create X and y [ /4 marks]
X = df.iloc[:,0:28] #tedious
X = df.drop('area', axis = 'columns')
y = df.area

# Use train_test_split on X, y [ /2 marks]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.5, random_state=0)


## Question 1.5 [ _ / 4 marks]

For our first model let us consider fitting a line to the data using linear regression. Use an sklearn model pipeline. For more information on Pipelines and how to implement them, feel free to take a look at the following link: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [6]:
# Create a pipeline for model 1 (M1) [ /4 marks]
M1 = Pipeline([
    ('lr1', LinearRegression())
])

## Question 1.6 [ _ / 6 marks]

For our second model let's add quadratic terms for all features (use `PolynomialFeatures`). Create a model pipeline for our second model (M2).

In [7]:
# Create a pipeline for model 2 (M2); you can set "include_bias=False" [ / 6 marks]
M2 = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('lr2', LinearRegression())
])

## Question 1.7 [ _ / 10 marks]

`temperature (temp)` and `relative humidity (RH)` may be important features to consider. Let's extend model 1 by adding a *squared* term for temp and a *cubed* term for RH. Before creating a pipeline for this model, we need a custom transformer: here we can specify a column for the squared temp and cubed RH terms. The transformer has been initialized below, but you'll need to finish it with 1-2 lines of code. After this, create your corresponding pipeline (M3).

In [8]:
# Modify the transform method of the KeyFeatures class [ /4 marks]
class KeyFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.assign(temp= X.temp**2)
        X = X.assign(RH= X.RH**3)
        return X

# Create a pipeline for model 3 (M3) [ /6 marks]
M3 = Pipeline([
    ('Key_Features', KeyFeatures()),
    ('lr3', LinearRegression())
])

## Question 1.8 [ _ /7 marks]

For models 1-3, use 4-fold Cross-validation (CV) on the training set and compute (and print) the mean CV score. Use `mse` as your error function.

In [9]:
# Use 4-fold CV on all models, print the mean CV scores [ /7 marks]
sc = make_scorer(mse)
kf = KFold(n_splits=4, shuffle=False)
print(f"CV loss (M1): {cross_val_score(M1, Xtrain, ytrain, cv=kf, scoring=sc).mean()}")
print(f"CV loss (M2): {cross_val_score(M2, Xtrain, ytrain, cv=kf, scoring=sc).mean()}")
print(f"CV loss (M3): {cross_val_score(M3, Xtrain, ytrain, cv=kf, scoring=sc).mean()}")

CV loss (M1): 2.1530199010134936
CV loss (M2): 95624.90775635651
CV loss (M3): 2.165843590568246


## Question 1.9 [ _ / 5 marks]

Which model would you choose at this stage? Why would you pick this model over the other models? Provide your response in a sentence or two.

**Your answer**: Based on the above values, M1: 2.153, M2: 95624, and M3: 2.165. Since we have to minimise the error, M1 model is the better fit as it has the lowest validation error.

## Question 2.1 [ _ /  7 marks]

Estimate the performance (test loss) of your chosen model on the test data (which has been held out until this point) using MSE. Print the test loss.

In [10]:
# Compute the test loss on the unseen (test) dataset [ /6 marks]
ypred = M1.fit(Xtrain, ytrain).predict(Xtest)
#trloss = mse(ytrain,M1.predict(Xtrain))
teloss = mse(ytest,M1.predict(Xtest))
print(f"Test loss: {teloss}")

Test loss: 2.102764237510351


## Question 2.2 [ _ /12 marks]

With your chosen model, compute (and print) a 95% confidence interval for the average test error using the Central Limit Theorem. Use the following formula: 

$$ \bar{L_n} \pm 1.96 * \frac{\sigma_{l}}{\sqrt{n}}$$

Here $\bar{L_n}$ is the average test loss (i.e. for our test set), $\sigma_l$ is the standard deviation (of our test losses), and $n$ is the total number of test losses we compute.  

In [11]:
# Compute the average test loss for our test set [ /6 marks]
std_err = np.std((ytest-ypred)*2,ddof=1)/np.sqrt(len(ypred))

# Calculate the 95% Confidence Interval for average test loss [ /6 marks]
cit=[teloss-1.96*std_err,teloss+1.96*std_err]

print(f"Central Limit Theorem: {cit}")

Central Limit Theorem: [1.7498196789849498, 2.455708796035752]
